In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

# Data step
groups = ['Schizophrenia Unspecified', 'First Episode Schizophrenia',
          'Schizophrenia Remission', 'Schizophrenia Without Remission', 'Combined']
means = np.array([2.27, 1.94, 2.03, 2.43])
sds = np.array([1.43, 1.09, 1.17, 1.23])
ns = np.array([9148, 648, 3019, 1256])
control_mean = 1.62
control_sd = 0.78
control_n = 1150

# Combined mean and SD calculation
combined_mean = np.sum(ns * means) / np.sum(ns)
variance_components = ns * (sds**2 + (means - combined_mean)**2)
combined_sd = np.sqrt(np.sum(variance_components) / np.sum(ns))
total_n = np.sum(ns)

# Add combined data to arrays
means = np.append(means, combined_mean)
ns = np.append(ns, total_n)

# Calculating diffrences in means
differences = means - control_mean

# Calculating SE
se = np.sqrt((sds**2 / ns[:4]) + (control_sd**2 / control_n))
combined_se = np.sqrt((combined_sd**2 / total_n) + (control_sd**2 / control_n))
se = np.append(se, combined_se)

# Calculating confidence intervals
ci_lower = differences - 1.96 * se
ci_upper = differences + 1.96 * se

# Calculating z-score and p-value for combined vs control
z_score = differences[-1] / se[-1]
p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))

# Creating the plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plotting data points and lines
for i, (diff, lower, upper) in enumerate(zip(differences, ci_lower, ci_upper)):
    ax.plot([lower, upper], [i, i], 'b-')
    ax.plot(diff, i, 'bo', markersize=10)

# Customizing the plot
ax.set_yticks(range(len(groups)))
ax.set_yticklabels(groups)
ax.set_xlabel('Mean Difference in NLR (Schizophrenia - Control)')
ax.set_title('Forest Plot of NLR Differences')

# Adding a vertical line at x=0
ax.axvline(x=0, color='gray', linestyle='--')

# Adding text for sample sizes
for i, n in enumerate(ns):
    ax.text(0.98, i, f'n = {n}', verticalalignment='center', transform=ax.get_yaxis_transform())

# Setting x-axis limits
ax.set_xlim(-0.2, 1.0)

# Adding legend
ax.plot([], [], 'bo-', label='Mean difference and 95% CI')
ax.legend(loc='lower right')

plt.tight_layout()
plt.savefig('forest_plot.png')
plt.show()

print(f"Combined Mean: {combined_mean:.2f}")
print(f"Combined SD: {combined_sd:.2f}")
print(f"Combined SE: {combined_se:.4f}")
print(f"Z-score (Combined vs Control): {z_score:.4f}")
print(f"P-value (Combined vs Control): {p_value:.4e}")
print(f"Effect Size (Mean Difference): {differences[-1]:.4f}")